# Running faster your code

* [Discrete signal energy](https://en.wikipedia.org/wiki/Energy_(signal_processing) can be computed as a particular case of the [dot product](https://en.wikipedia.org/wiki/Dot_product) when both signals are the same:
$$ ~\\ E_{s} \ \ = \ \ \langle x(n), x(n)\rangle \ \ = \sum_{n}{|x(n)|^2} = \sum_{n}{x(n)y(n)}$$
Let's see how to speed this operation.

## 1 Vectorize

In [ ]:
import numpy as np

def non_vectorized_dot_product(x, y):
    """Return the sum of x[i] * y[j] for all pairs of indices i, j.

    Example:
    
        >>> my_dot_product(np.arange(20), np.arange(20))
    
    """
    result = 0
    for i in range(len(x)):
        result += x[i] * y[i]
    return result

signal = np.random.random(1000)
#print(signal)

In [ ]:
%timeit non_vectorized_dot_product(signal, signal)

In [ ]:
non_vectorized_dot_product(signal, signal)

Now, using Numpy's array multiplication and sum:

In [ ]:
%timeit np.sum(signal*signal)

In [ ]:
np.sum(signal*signal)

* Another example to see that vectorization not only involves pure computation:

In [ ]:
# https://softwareengineering.stackexchange.com/questions/254475/how-do-i-move-away-from-the-for-loop-school-of-thought
def cleanup(x, missing=-1, value=0):
    """Return an array that's the same as x, except that where x ==
    missing, it has value instead.

    >>> cleanup(np.arange(-3, 3), value=10)
    ... # doctest: +NORMALIZE_WHITESPACE
    array([-3, -2, 10, 0, 1, 2])

    """
    result = []
    for i in range(len(x)):
        if x[i] == missing:
            result.append(value)
        else:
            result.append(x[i])
    return np.array(result)

array = np.arange(-8,8)
print(array)
print(cleanup(array, value=10, missing=0))

In [ ]:
array = np.arange(-1000,1000)
%timeit cleanup(array, value=10, missing=0)
print(array[995:1006])
print(cleanup(array, value=10, missing=0)[995:1006])

In [ ]:
# http://www.secnetix.de/olli/Python/list_comprehensions.hawk
# https://docs.python.org/3/library/functions.html#zip
value = [10]*2000
%timeit [xv if c else yv for (c,xv,yv) in zip(array == 0, value, array)]
print([xv if c else yv for (c,xv,yv) in zip(array == 0, value, array)][995:1006])

In [ ]:
# https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.where.html
%timeit np.where(array == 0, 10, array)
print(np.where(array == 0, 10, array)[995:1006])

* Another example extracted from this [HPC tutorial](https://github.com/pyHPC/pyhpc-tutorial):

In [ ]:
from math import sin
import numpy as np

arr = np.arange(1000000)
%timeit [sin(i)**2 for i in arr]

In [ ]:
%timeit np.sin(arr)**2

## 2 Use in-place operations

In [ ]:
a = np.random.random(500000)
print(a[0:10])
b = np.copy(a)
%timeit global a; a = 10*a
a = 10*a
print(a[0:10])

In [ ]:
a = np.copy(b)
print(a[0:10])
%timeit global a ; a *= 10
a *= 10
print(a[0:10])

## 3 Maximize locality in memory acess

In [ ]:
a = np.random.rand(100,50)
b = np.copy(a)

In [ ]:
def mult(x, val):
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i][j] /= val
%timeit -n 1 -r 1 mult(a, 10)

In [ ]:
a = np.copy(b)

def mult2(x, val):
    for j in range(x.shape[1]):
        for i in range(x.shape[0]):
            x[i][j] /= val
            
%timeit -n 1 -r 1 mult2(a, 10)

In [ ]:
# http://www.scipy-lectures.org/advanced/optimizing/
# https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.sum.html

In [ ]:
c = np.zeros((1000, 1000), order='C')

In [ ]:
%timeit c.sum(axis=0)
c.sum(axis=0).shape

In [ ]:
%timeit c.sum(axis=1)
c.sum(axis=1).shape

## 4 Delegate in C
When you want to speed-up your code or simply when you need to reuse C code, it is possible to use it from Python. There are several alternatives:

1. [Cython](http://cython.org/): A superset of Python to allow you call C functions and load Python variables with C ones. 
2. [SWIG (Simplified Wrapper Interface Generator)](http://www.swig.org/): A software development tool to connect C/C++ programs with other languages (included Python).
3. [Ctypes](http://python.net/crew/theller/ctypes/): A Python package that can be used to call shared libraries (`.ddl`/`.so`/`.dylib`) from Python.
4. [Python-C-API](https://docs.python.org/3.6/c-api/index.html): A low-level interface between (compiled) C code and Python.

A function to optimize:

In [ ]:
!cat sum_array_lib.py

In [ ]:
# Please, restart the kernel to ensure that the module sum_array_lib is re-loaded
!rm -f sum_array_lib.cpython*.so
import sum_array_lib
import array as arr
a = arr.array('d', [i for i in range(100000)])
#a = [1 for i in range(100000)]
%timeit sum_array_lib.sum_array(a, len(a))
sum = sum_array_lib.sum_array(a, len(a))
print(sum)

### 4.1 Cython

[Python with C data types.](https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html)

#### Working flow:
```
      Cython compiler        C compiler
.pyx -----------------> .c --------------> .so
```

#### Installation
$ pip install Cython

#### Compilation of pure Python code:

In [ ]:
!cp sum_array_lib.py sum_array_lib.pyx

In [ ]:
!cat sum_array_lib.pyx

In [ ]:
!cat Cython/basic/setup.py

In [ ]:
!rm -f sum_array_lib.cpython*.so
!python Cython/basic/setup.py build_ext --inplace

In [ ]:
# Please, restart the kernel to ensure that the module sum_array_lib is re-loaded
import sum_array_lib
import array as arr
a = arr.array('d', [i for i in range(100000)])
#a = [1.1 for i in range(100000)]
%timeit sum_array_lib.sum_array(a, len(a))
sum = sum_array_lib.sum_array(a, len(a))
print(sum)

#### Defining C types:

In [ ]:
!cat Cython/cdef/sum_array_lib.pyx

In [2]:
!cat Cython/cdef/setup.py

from distutils.core import setup
from Cython.Build import cythonize
import numpy

setup(
    ext_modules=cythonize("Cython/cdef/sum_array_lib.pyx"),
    include_dirs=[numpy.get_include()]
)


In [ ]:
# Please, restart the kernel to ensure that the module sum_array_lib is re-loaded
!rm sum_array_lib.cpython*.so
!python Cython/cdef/setup.py build_ext --inplace

In [1]:
# Please, restart the kernel to ensure that the module sum_array_lib is re-loaded
import array as arr
import sum_array_lib
#import numpy as np
#a = np.arange(100000)
a = arr.array('d', [i for i in range(100000)])
%timeit sum_array_lib.sum_array(a, len(a))
print(sum)

164 µs ± 2.28 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
<built-in function sum>


### 4.2 Python-C

Python-C-API is the most flexible and efficient alternative, but also the hardest to code.

#### The C code to reuse in Python

In [3]:
!cat sum_array_lib.c

/* Generated by Cython 0.29.2 */

/* BEGIN: Cython Metadata
{
    "distutils": {
        "name": "sum_array_lib",
        "sources": [
            "sum_array_lib.pyx"
        ]
    },
    "module_name": "sum_array_lib"
}
END: Cython Metadata */

#define PY_SSIZE_T_CLEAN
#include "Python.h"
#ifndef Py_PYTHON_H
    #error Python headers needed to compile C extensions, please install development version of Python.
#elif PY_VERSION_HEX < 0x02060000 || (0x03000000 <= PY_VERSION_HEX && PY_VERSION_HEX < 0x03030000)
    #error Cython requires Python 2.6+ or Python 3.3+.
#else
#define CYTHON_ABI "0_29_2"
#define CYTHON_HEX_VERSION 0x001D02F0
#define CYTHON_FUTURE_DIVISION 0
#include <stddef.h>
#ifndef offsetof
  #define offsetof(type, member) ( (size_t) & ((type*)0) -> member )
#endif
#if !defined(WIN32) && !defined(MS_WINDOWS)
  #ifndef __stdcall
    #define __stdcall
  #endif
  #ifndef __cdecl
    #define __cdecl
  #endif
  #ifndef __fastcall
    #define __fastcall
  #endif
#endif
#ifndef DL_

In [ ]:
!cat sum_array.c

In [ ]:
!gcc -O3 sum_array.c -o sum_array
!./sum_array

### The module

In [ ]:
!cat sum_array_module.c

### Module compilation

In [ ]:
!cat setup.py

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
import sum_array_module
import numpy as np
a = np.arange(100000)
%timeit sum_array_module.sumArray(a)
print(sum)

However, remember: vectorize when possible!

In [ ]:
%timeit np.sum(a)
print(sum)